In [12]:
# cv_analysis.ipynb

import json
import os
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader


PROJECT_ROOT = Path("..").resolve()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.data.dataloader import (
    ForgeryDataset,
    detection_collate_fn,
    get_val_transform,
)
from src.models.mask2former_v2 import Mask2FormerForgeryModel
from src.utils.config_utils import load_yaml, sanitize_model_kwargs


In [15]:
# -----------------
# Constants / Paths
# -----------------
OOF_ROOT = PROJECT_ROOT / "experiments" / "oof_results"
FULL_TRAIN_ROOT = PROJECT_ROOT / "experiments" / "full_train_results"

SWEEP_RUN_DIRS = [
    PROJECT_ROOT / "experiments" / "qscore_sweep_v2",   # <- your latest run
    # PROJECT_ROOT / "experiments" / "qscore_sweep_v1",
]

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CFG_PATH = PROJECT_ROOT / "config" / "base_v2.yaml" 


def load_sweep_runs(run_dirs):
    rows = []
    for d in run_dirs:
        d = Path(d)
        summary_path = d / "summary.json"
        if not summary_path.exists():
            continue
        run_rows = json.loads(summary_path.read_text())
        for r in run_rows:
            r = dict(r)
            r["run_dir"] = str(d)
            # normalize csv to absolute path (script stores it as a string path)
            r["csv_path"] = str((PROJECT_ROOT / r["csv"]).resolve()) if isinstance(r.get("csv"), str) else None
            rows.append(r)
    df = pd.DataFrame(rows)
    if len(df) == 0:
        return df
    # convenient sort: best score first
    df = df.sort_values(["score"], ascending=False).reset_index(drop=True)
    return df


sweep_df = load_sweep_runs(SWEEP_RUN_DIRS)
display(sweep_df.head(20))


,mask_threshold,qscore_threshold,topk,min_mask_mass,presence_threshold,area_threshold,score,csv,run_dir,csv_path
0,0.5,NaN,1.0,0.002,0.02,NaN,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
1,0.5,NaN,1.0,0.002,0.02,0.002,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
2,0.5,NaN,1.0,0.002,0.02,0.001,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
3,0.5,NaN,1.0,0.002,0.02,0.002,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p01_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
4,0.5,NaN,1.0,0.002,0.02,NaN,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p01_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
5,0.5,NaN,1.0,NaN,0.02,0.002,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
6,0.5,NaN,1.0,NaN,0.02,0.001,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
7,0.5,NaN,1.0,NaN,0.02,NaN,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
8,0.5,NaN,1.0,0.001,0.02,0.002,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
9,0.5,NaN,1.0,0.001,0.02,0.001,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...


In [16]:
# Quick summary stats for the most recent sweep run

assert len(SWEEP_RUN_DIRS) > 0
latest_run = SWEEP_RUN_DIRS[0]

df = sweep_df[sweep_df["run_dir"] == str(latest_run)].copy()

print(f"Run dir: {latest_run}")
print(f"Total configs: {len(df)}")
print(f"Best score: {df['score'].max():.5f}")
print(f"Median score: {df['score'].median():.5f}")
print(f"Worst score: {df['score'].min():.5f}")

display(
    df.head(10)[
        [
            "score",
            "qscore_threshold",
            "topk",
            "min_mask_mass",
            "presence_threshold",
            "area_threshold",
        ]
    ]
)

# Optional: aggregated view by qscore threshold
display(
    df.groupby("qscore_threshold")["score"]
      .agg(["count", "max", "mean"])
      .sort_values("max", ascending=False)
)


Run dir: C:\Users\piiop\Desktop\Portfolio\Projects\RecodAI_LUC\experiments\qscore_sweep_v2
Total configs: 432
Best score: 0.51175
Median score: 0.46160
Worst score: 0.34161


,score,qscore_threshold,topk,min_mask_mass,presence_threshold,area_threshold
0,0.511746,NaN,1.0,0.002,0.02,NaN
1,0.511746,NaN,1.0,0.002,0.02,0.002
2,0.511746,NaN,1.0,0.002,0.02,0.001
3,0.511746,NaN,1.0,0.002,0.02,0.002
4,0.511746,NaN,1.0,0.002,0.02,NaN
5,0.511746,NaN,1.0,NaN,0.02,0.002
6,0.511746,NaN,1.0,NaN,0.02,0.001
7,0.511746,NaN,1.0,NaN,0.02,NaN
8,0.511746,NaN,1.0,0.001,0.02,0.002
9,0.511746,NaN,1.0,0.001,0.02,0.001


,count,max,mean
qscore_threshold,,,
0.01,27,0.511745,0.475718
0.02,27,0.511745,0.511745
0.03,27,0.485874,0.485874
0.05,27,0.463070,0.463070


In [17]:
# Analyze duplicate / invariant scores in the most recent sweep run

latest_run = SWEEP_RUN_DIRS[0]
df = sweep_df[sweep_df["run_dir"] == str(latest_run)].copy()

# ---- unique score distribution ----
score_counts = (
    df.groupby("score")
      .size()
      .reset_index(name="count")
      .sort_values(["count", "score"], ascending=[False, False])
)

print(f"Total configs: {len(df)}")
print(f"Unique scores: {score_counts.shape[0]}")
print(f"Configs per unique score (top 10):")
display(score_counts.head(10))

# ---- inspect knobs that vary while score stays constant ----
TARGET_SCORE = 0.511745

same_score = df[np.isclose(df["score"], TARGET_SCORE)]

print(f"Configs with score={TARGET_SCORE}: {len(same_score)}")

display(
    same_score[
        [
            "qscore_threshold",
            "topk",
            "min_mask_mass",
            "presence_threshold",
            "area_threshold",
        ]
    ]
    .sort_values(
        [
            "qscore_threshold",
            "topk",
            "min_mask_mass",
            "presence_threshold",
            "area_threshold",
        ]
    )
)

# ---- how much each knob actually influences the score ----
def knob_variation(df, knob):
    g = df.groupby(knob)["score"]
    return pd.DataFrame(
        {
            "unique_scores": g.nunique(),
            "min_score": g.min(),
            "max_score": g.max(),
            "range": g.max() - g.min(),
        }
    ).sort_values("range", ascending=False)


knob_stats = {
    "qscore_threshold": knob_variation(df, "qscore_threshold"),
    "topk": knob_variation(df, "topk"),
    "min_mask_mass": knob_variation(df, "min_mask_mass"),
    "presence_threshold": knob_variation(df, "presence_threshold"),
    "area_threshold": knob_variation(df, "area_threshold"),
}

for k, v in knob_stats.items():
    print(f"\nKnob: {k}")
    display(v)


Total configs: 432
Unique scores: 24
Configs per unique score (top 10):


,score,count
22,0.511745,72
23,0.511746,36
21,0.511745,36
20,0.485874,27
19,0.463070,27
16,0.461600,24
13,0.458731,24
10,0.452779,24
8,0.372724,24
6,0.360666,24


Configs with score=0.511745: 144


,qscore_threshold,topk,min_mask_mass,presence_threshold,area_threshold
118,0.01,NaN,0.001,0.02,0.001
119,0.01,NaN,0.001,0.02,0.002
117,0.01,NaN,0.001,0.02,NaN
115,0.01,NaN,0.002,0.02,0.001
116,0.01,NaN,0.002,0.02,0.002
...,...,...,...,...,...
92,NaN,3.0,NaN,0.02,0.002
44,NaN,3.0,NaN,0.02,NaN
53,NaN,3.0,NaN,0.02,NaN
87,NaN,3.0,NaN,0.02,NaN



Knob: qscore_threshold


...

,unique_scores,min_score,max_score,range
min_mask_mass,,,,
0.001,20,0.341806,0.511746,0.169940
0.002,20,0.341999,0.511746,0.169746



Knob: presence_threshold


,unique_scores,min_score,max_score,range
presence_threshold,,,,
0.01,12,0.452779,0.511745,0.058966
0.02,5,0.463070,0.511746,0.048676



Knob: area_threshold


,unique_scores,min_score,max_score,range
area_threshold,,,,
0.001,10,0.360666,0.511746,0.151080
0.002,10,0.372724,0.511746,0.139022


In [18]:
import numpy as np
import pandas as pd

latest_run = SWEEP_RUN_DIRS[0]
df = sweep_df[sweep_df["run_dir"] == str(latest_run)].copy()

BEST_SCORE = df["score"].max()
tie = df[np.isclose(df["score"], BEST_SCORE)].copy()
print(f"Best score: {BEST_SCORE:.6f} | tied configs: {len(tie)}")
display(tie.head(10))

knobs = ["qscore_threshold", "topk", "min_mask_mass", "presence_threshold", "area_threshold"]

# (1) per-knob frequency within the tie plateau
freq_tables = {}
for k in knobs:
    freq = tie[k].value_counts(dropna=False).reset_index()
    freq.columns = [k, "count"]
    freq["pct"] = freq["count"] / len(tie)
    freq_tables[k] = freq
    print(f"\n{k} distribution among ties:")
    display(freq)

# (2) choose mode per knob
modes = {k: freq_tables[k].iloc[0][k] for k in knobs}
print("\nModes among ties:", modes)

# (3) pick the most "central" config: maximize matches to mode
# Tie-break: prefer simpler (topk smallest, thresholds disabled, min_mask_mass smallest)
def simplicity_key(row):
    # smaller is "simpler"
    return (
        float(row["topk"]),
        0 if float(row["presence_threshold"]) < 0 else 1,
        0 if float(row["area_threshold"]) < 0 else 1,
        float(row["min_mask_mass"]),
        float(row["qscore_threshold"]),
    )

tie["mode_matches"] = sum(tie[k] == modes[k] for k in knobs)

best_central = (
    tie.sort_values(
        by=["mode_matches"],
        ascending=[False],
        kind="mergesort",
    )
)

# now apply simplicity among those with max matches
max_matches = best_central["mode_matches"].max()
candidates = best_central[best_central["mode_matches"] == max_matches].copy()

candidates["_simplicity"] = candidates.apply(simplicity_key, axis=1)
chosen = candidates.sort_values("_simplicity", ascending=True).iloc[0]

print("\nChosen config (central + simple):")
display(chosen[["score"] + knobs + ["csv_path", "run_dir"]])

# optional: show top 20 tied configs by "centrality" then simplicity
candidates = candidates.sort_values("_simplicity", ascending=True)
display(candidates[["score"] + knobs + ["csv_path"]].head(20))


Best score: 0.511746 | tied configs: 144


,mask_threshold,qscore_threshold,topk,min_mask_mass,presence_threshold,area_threshold,score,csv,run_dir,csv_path
0,0.5,NaN,1.0,0.002,0.02,NaN,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
1,0.5,NaN,1.0,0.002,0.02,0.002,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
2,0.5,NaN,1.0,0.002,0.02,0.001,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
3,0.5,NaN,1.0,0.002,0.02,0.002,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p01_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
4,0.5,NaN,1.0,0.002,0.02,NaN,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p01_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
5,0.5,NaN,1.0,NaN,0.02,0.002,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
6,0.5,NaN,1.0,NaN,0.02,0.001,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
7,0.5,NaN,1.0,NaN,0.02,NaN,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
8,0.5,NaN,1.0,0.001,0.02,0.002,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
9,0.5,NaN,1.0,0.001,0.02,0.001,0.511746,experiments\qscore_sweep_v2\oof_mt0p5_qt0p05_k...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...



qscore_threshold distribution among ties:


,qscore_threshold,count,pct
0,NaN,108,0.7500
1,0.02,27,0.1875
2,0.01,9,0.0625



topk distribution among ties:


...


area_threshold distribution among ties:


,area_threshold,count,pct
0,NaN,48,0.333333
1,0.002,48,0.333333
2,0.001,48,0.333333



Modes among ties: {'qscore_threshold': np.float64(nan), 'topk': np.float64(1.0), 'min_mask_mass': np.float64(0.002), 'presence_threshold': np.float64(0.02), 'area_threshold': np.float64(nan)}

Chosen config (central + simple):


score                                                          0.511746
qscore_threshold                                                    NaN
topk                                                                1.0
min_mask_mass                                                     0.002
presence_threshold                                                 0.02
area_threshold                                                      NaN
csv_path              C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
run_dir               C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
Name: 0, dtype: object

,score,qscore_threshold,topk,min_mask_mass,presence_threshold,area_threshold,csv_path
0,0.511746,NaN,1.0,0.002,0.02,NaN,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
1,0.511746,NaN,1.0,0.002,0.02,0.002,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
2,0.511746,NaN,1.0,0.002,0.02,0.001,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
3,0.511746,NaN,1.0,0.002,0.02,0.002,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
4,0.511746,NaN,1.0,0.002,0.02,NaN,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
13,0.511746,NaN,1.0,0.002,0.02,0.001,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
27,0.511746,NaN,1.0,0.002,0.02,NaN,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
28,0.511746,NaN,1.0,0.002,0.02,0.001,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
29,0.511746,NaN,1.0,0.002,0.02,0.002,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
32,0.511746,NaN,1.0,0.002,0.02,NaN,C:\Users\piiop\Desktop\Portfolio\Projects\Reco...
